In [3]:
import pandas as pd
import openstudio
import os
import numpy as np
import sqlite3
from os import listdir
import plotly.express as px
import sqlite3
import timedelta
print('here')


here


In [ ]:
def create_osw(folder_path, outage_type, model_name, beginning_date, beginning_hour, duration, name_individual_simulation, output_dir_name):
#     outage_type = 'Power_and_Heat'
#     model_name = '14_Hangar'
    
#     folder_path = '/Users/cbianchi/Documents/GitHub/example_workflow/'
    measures_path = folder_path + 'measures'
    
    
    model_path = 'ERMA_Power_Outage/' + model_name + '/run/in.osm'
    model_save_path = folder_path + output_dir_name + name_individual_simulation + '/' + name_individual_simulation
    seed_file_name = name_individual_simulation + '.osm'
    new_osw_path = model_save_path + '.osw'
#     print(new_osw_path)

    os_script = f"""
    require 'openstudio'
    require 'fileutils'
    require 'pycall'
    
    # This is the path to the OpenStudio CLI
    # If you call this script using the CLI,
    # it will just return the path to the thing you called.
    cli = OpenStudio::getOpenStudioCLI

    # Setup a directory for all of the output
    #FileUtils.rm_rf('{output_dir_name}')
    #FileUtils.mkdir('{output_dir_name}')

    # The osw defines the measures to run
    osw = OpenStudio::WorkflowJSON.new('example.osw')
    osw.addMeasurePath('{measures_path}')

    # This example uses the built in OpenStudio example model, because it is convenient,
    # but model files could be globbed from a directory and loaded.
    model = OpenStudio::Model::exampleModel()

    # Load model building by building
    model = OpenStudio::Model::Model.load('{model_path}').get

    model.save(OpenStudio::toPath('{model_save_path}'), true)
    # Update the osw to point to the new model
    osw.setSeedFile('{seed_file_name}')
    
    # Set measure arguments
    measure = OpenStudio::MeasureStep.new('set_power_and_heat_off')
    measure.setArgument('otg_date','{beginning_date}')
    measure.setArgument('otg_hr','{beginning_hour}')
    measure.setArgument('otg_len','{duration}')
    measure.setArgument('outage_type','{outage_type}'.gsub('_', ' '))
    measure_type = OpenStudio::MeasureType.new('ModelMeasure')

    var_measure1 = OpenStudio::MeasureStep.new('AddOutputVariable')
    var_measure1.setArgument('variable_name','Zone Mean Air Temperature')
    var_measure1.setArgument('reporting_frequency','hourly')
    var_measure1.setArgument('key_value','*')
    measure_type = OpenStudio::MeasureType.new('ReportingMeasure')

    var_measure2 = OpenStudio::MeasureStep.new('AddOutputVariable')
    var_measure2.setArgument('variable_name','Zone Operative Temperature')
    var_measure2.setArgument('reporting_frequency','hourly')
    var_measure2.setArgument('key_value','*')
    measure_type = OpenStudio::MeasureType.new('ReportingMeasure')
    
    var_measure3 = OpenStudio::MeasureStep.new('AddOutputVariable')
    var_measure3.setArgument('variable_name','Zone Thermal Comfort Fanger Model PMV')
    var_measure3.setArgument('reporting_frequency','hourly')
    var_measure3.setArgument('key_value','*')
    measure_type = OpenStudio::MeasureType.new('ReportingMeasure')
    
    var_measure4 = OpenStudio::MeasureStep.new('AddOutputVariable')
    var_measure4.setArgument('variable_name','Zone Thermal Comfort Fanger Model PPD')
    var_measure4.setArgument('reporting_frequency','hourly')
    var_measure4.setArgument('key_value','*')
    measure_type = OpenStudio::MeasureType.new('ReportingMeasure')
    
    
    osw.setWorkflowSteps([measure, var_measure1, var_measure2,  var_measure3,  var_measure4])

    # Save the osw to a unique location
    osw.saveAs(OpenStudio::toPath('{new_osw_path}'))
    """
    cmd = '/Users/cbianchi/Documents/GitHub/example_workflow/OpenStudio-3.6.1+bb9481519e-Darwin-arm64/bin/openstudio -e "{}"'.format(os_script)
    s = os.popen(cmd).read()


def run_osw(osw_path):
    os.system('openstudio run -w ' + osw_path)
    
    

def df_query(query, sql_dir):
    '''opens sql, makes query (native sql), closes sql and returns df'''
    conn = sqlite3.connect(sql_dir + 'eplusout.sql' )
    df = pd.read_sql(query, conn)
    conn.close()
    return df

def filter_tabular(df, filterquery):
    '''search available tabulardata for any string, return dataframe'''
    lines = df[df.apply(lambda row: row.astype(str).str.contains(filterquery).any(), axis=1)]
    return lines

def export_results_to_csv(sql_dir, simulation_name, save_path):

    variables = ['Electricity:Facilit',
     'NaturalGas:Facility',
     'Zone Mean Air Temperature',
     'Zone Thermal Comfort ASHRAE 55 Adaptive Model Running Average Outdoor Air Temperature',
     'Zone Operative Temperature',
     'Zone Thermal Comfort Fanger Model PMV',
     'Zone Thermal Comfort Fanger Model PPD']

    #List of all the available variables
    table_variables = df_query("SELECT * FROM ReportDataDictionary WHERE ReportingFrequency = 'Hourly'", sql_dir)
    
#     print(table_variables)
    
    
    #create empty dataframe
    
    df_final = pd.DataFrame(index=pd.Index(pd.date_range(start='2021-01-01 01:00:00', end='2022-01-01 00:59:00', freq='60min'), name='date_time'))
#     df_final.index = 
    #Pinpoint individual variable and add it to the results DF
    for var in variables:
        var_names = filter_tabular(table_variables, var)['KeyValue'].values
        indexes = filter_tabular(table_variables, var)['ReportDataDictionaryIndex'].values
#         print(indexes)
        for i in range(0,len(indexes)):
    #         print(val)
            column = df_query('SELECT "Value","ReportDataDictionaryIndex","TimeIndex" FROM "ReportData" WHERE "ReportDataDictionaryIndex"='+ str(indexes[i]), sql_dir)
#             print(column)
#             column['Value']
            try:
                name = var + '-' + var_names[i]
            except TypeError:
                name = var
            df_final[name] = column['Value'].to_list()
            
#     print(df_final)
    df_final.to_csv(save_path + simulation_name + '.csv')    
    
    
def run_individual_simulation(outage_type, model_name, beginning_date,beginning_h, duration, save_path, output_dir_name, name_individual_simulation):
    folder_path = '/Users/cbianchi/Documents/GitHub/example_workflow/'
    # output_dir_name = 'output_ERMA/'
    # save_path = 'results_CSVs/'
    
    osw_path = folder_path + output_dir_name + name_individual_simulation + '/' + name_individual_simulation + '.osw'
    sql_dir = folder_path + output_dir_name + name_individual_simulation + '/run/'


    #Start with creating the OSW in the right folder
    create_osw(folder_path, outage_type, model_name, beginning_date, str(beginning_h), str(duration), name_individual_simulation, output_dir_name)

    # Run the OS Simulation
    run_osw(osw_path)
    # print(sql_dir)
    # Open the results, process them, export to CSV
    export_results_to_csv(sql_dir, name_individual_simulation, save_path)
    
    
#=========================================
#Workflow individual simulation
#=========================================  

outage_list = pd.read_csv('outage_list_NEW.csv')
save_path = 'results_CSVs_NEW/'
output_dir_name = 'ERMA_simulations_NEW/'

monitored_quantity = 'PMV'


for i in range(0, len(outage_list)):
    
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ ' + str(i))
    model = outage_list[outage_list.index==i]['model'].values[0]
    outage_type = outage_list[outage_list.index==i]['outage_type'].values[0]
    outage_date_ = outage_list[outage_list.index==i]['outage_date'].values[0]
    duration_hours = outage_list[outage_list.index==i]['duration_hours'].values[0]
    start_time = outage_list[outage_list.index==i]['start time'].values[0]
    scenario = outage_list[outage_list.index==i]['scenario'].values[0]
    event = outage_list[outage_list.index==i]['event'].values[0]
    
    #Reformat sart_time
    start_time = pd.to_datetime(start_time).round("H").hour
    
    #Process outage date
    start_date = pd.to_datetime(outage_date_ + '2021', format='%d-%b%Y') + pd.Timedelta(hours=start_time, minutes=0)
    outage_date = start_date.month_name() + ' ' + str(start_date.day)

    
    # check if we are across 2 years
    
    end_date = start_date + pd.Timedelta(hours=duration_hours, minutes=0)

    if end_date > pd.to_datetime('2021-12-31 23:59:00'):
        #Split Simulation in 2
        start_date_1 = pd.to_datetime(start_date)
        end_date_1 = pd.to_datetime('2021-12-31 23:59:00')
        duration_hours_1 = timedelta.Timedelta(end_date_1 - start_date_1).total.hours
        outage_date_1 = outage_date
        start_time_1 = start_time
        #Individual simulation name
        name_individual_simulation = outage_type + '_' + model + '_' + outage_date.replace(' ', '_') + '_Y1'
        print(scenario + '   ===========   ' + model + ' - ' + outage_type + ' - ' + outage_date + ' Y1')
        #Run simulation
        run_individual_simulation(outage_type, model, outage_date_1, start_time_1, duration_hours_1, save_path + str(scenario) + '/', output_dir_name + str(scenario) + '/', name_individual_simulation)
        #Open Results
        outage_df = pd.read_csv(save_path + str(scenario) + '/' + name_individual_simulation + '.csv', index_col = 'date_time', parse_dates = True)
        outage_df = outage_df[(outage_df.index >= start_date_1) & (outage_df.index <= end_date_1)]
        filtered_columns = list(filter(lambda x: any(substring in x for substring in [monitored_quantity]), outage_df.columns))
        df_out_mean_y1 = outage_df[filtered_columns].mean(axis=1).to_frame()
        
        
        #Second chunk
        start_date_2 = pd.to_datetime('2021-01-01 00:00:00')
        duration_hours_2 = duration_hours - duration_hours_1- 1
        end_date_2 = start_date_2 + pd.Timedelta(hours=duration_hours_2, minutes=0)        
        outage_date_2 = 'January 1'
        start_time_2 = '0'
        #Individual simulation name
        name_individual_simulation = outage_type + '_' + model + '_' + outage_date.replace(' ', '_') + '_Y2'
        print(scenario + '   ===========   ' + model + ' - ' + outage_type + ' - ' + outage_date + ' Y2')
        #Run simulation
        run_individual_simulation(outage_type, model, outage_date_2, start_time_2, duration_hours_2, save_path + str(scenario) + '/', output_dir_name + str(scenario) + '/', name_individual_simulation)
        #Open Results
        outage_df = pd.read_csv(save_path + str(scenario) + '/' + name_individual_simulation + '.csv', index_col = 'date_time', parse_dates = True)
        outage_df = outage_df[(outage_df.index >= start_date_2) & (outage_df.index <= end_date_2)]
        filtered_columns = list(filter(lambda x: any(substring in x for substring in [monitored_quantity]), outage_df.columns))
        df_out_mean_y2 = outage_df[filtered_columns].mean(axis=1).to_frame()
        
        #Save
        df_out_mean = df_out_mean_y1.append(df_out_mean_y2, ignore_index=False)

    else:
        #Individual simulation name
        name_individual_simulation = outage_type + '_' + model + '_' + outage_date.replace(' ', '_')

        print(scenario + '   ===========   ' + model + ' - ' + outage_type + ' - ' + outage_date)
        #Run simulation
        run_individual_simulation(outage_type, model, outage_date, start_time, duration_hours, save_path + str(scenario) + '/', output_dir_name + str(scenario) + '/', name_individual_simulation)

        #Open Results
        outage_df = pd.read_csv(save_path + str(scenario) + '/' + name_individual_simulation + '.csv', index_col = 'date_time', parse_dates = True)

        outage_df = outage_df[(outage_df.index >= start_date) & (outage_df.index <= end_date)]
        filtered_columns = list(filter(lambda x: any(substring in x for substring in [monitored_quantity]), outage_df.columns))
        df_out_mean = outage_df[filtered_columns].mean(axis=1).to_frame()

        
    #Individual simulation name
    name_individual_simulation = outage_type + '_' + model + '_' + outage_date.replace(' ', '_')
    
    df_out_mean = df_out_mean.rename(columns={df_out_mean.columns[0]: "Average PMV"})
    df_out_mean.loc[df_out_mean['Average PMV'] <= -3, 'Habitability?'] = 'NH' 
    df_out_mean.loc[df_out_mean['Average PMV'] > -3, 'Habitability?'] = 'H' 
    df_out_mean.to_csv('outages_CSVs_NEW/' + str(scenario)+ '/' + name_individual_simulation + '.csv')









@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 0
tsunami   ===========   N2 - Power_and_Heat - March 16
[utilities.idf.IdfObject] <1> IdfObject of type 'OS:ZoneAirHeatBalanceAlgorithm' cannot have field index of 2. Cutting off IdfObject field parsing here, with the following text remaining: 

  ;                                       !- Do Space Heat Balance for Simulation
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 1
tsunami   ===========   15_Hangar - Power_and_Heat - March 16
[utilities.idf.IdfObject] <1> IdfObject of type 'OS:ZoneAirHeatBalanceAlgorithm' cannot have field index of 2. Cutting off IdfObject field parsing here, with the following text remaining: 

  ;                                       !- Do Space Heat Balance for Simulation
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 2
tsunami   ===========   20_Hangar - Power_and_Heat - March 16
[utilities.idf.IdfObject] <1> IdfObject of type 'OS:ZoneAirHeatBalanceAlgorithm' cannot have field index of 2. Cutting off IdfObject field parsing here, with

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 22
KEA_loss   ===========   20_Hangar - Power_and_Heat - February 4
[utilities.idf.IdfObject] <1> IdfObject of type 'OS:ZoneAirHeatBalanceAlgorithm' cannot have field index of 2. Cutting off IdfObject field parsing here, with the following text remaining: 

  ;                                       !- Do Space Heat Balance for Simulation
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 23
KEA_loss   ===========   14_Hangar - Power_and_Heat - February 4
[utilities.idf.IdfObject] <1> IdfObject of type 'OS:ZoneAirHeatBalanceAlgorithm' cannot have field index of 2. Cutting off IdfObject field parsing here, with the following text remaining: 

  ;                                       !- Do Space Heat Balance for Simulation
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 24
KEA_loss   ===========   N96_2020 - Power_and_Heat - February 4
[utilities.idf.IdfObject] <1> IdfObject of type 'OS:ZoneAirHeatBalanceAlgorithm' cannot have field index of 2. Cutting off IdfObject field 

# Downselect Outage

In [ ]:
test = pd.read_csv('results_CSVs/Power_and_Heat_MWD_August_1.csv', index_col = 'date_time', parse_dates = True)
# test

In [ ]:
# list_models = ['15_Hangar']
# outage_types = ['Power_and_Heat', 'Heat_only']
# outage_types = ['Power_and_Heat']
# outage_dates = ['January 2', 'May 5']
# outage_dates = ['February 10']
beginning_date = 'August 1'
# beginning_hour = 3
# duration = 25
# if beginning_hour<10:
#     hour_start = '0' + str(beginning_hour)
# else:
#     hour_start = str(beginning_hour)

In [ ]:
# pd.to_datetime(beginning_date + '2021' + hour_start + ':00', format='%B %d%Y%H:M')
start_date = pd.to_datetime(beginning_date + '2021', format='%B %d%Y') + pd.Timedelta(hours=beginning_hour-1, minutes=0)

end_date = start_date + pd.Timedelta(hours=duration+2, minutes=0)
outage_df = test[(test.index >= start_date) & (test.index <= end_date)]
filtered_columns = list(filter(lambda x: any(substring in x for substring in ['PMV']), test.columns))
outage_df[filtered_columns]
# pd.to_datetime(beginning_date + ' 2021', format='%B %d %Y')

In [ ]:
start_date


In [ ]:
end_date

In [ ]:
# test.columns[]
